# WeatherPy - *FINAL*

In [26]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import json
import scipy.stats as st
from datetime import date
from config import weather_api_key
from config import g_key
from citipy import citipy
import csv

In [3]:
url = url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f'{url}appid={weather_api_key}&units={units}&q='

## Generate Cities List

In [4]:
# Testing citipy nearest_city and country_code
city = citipy.nearest_city(-55.67, 120.2)
city.city_name
city.country_code

'au'

In [5]:

rand_lat = np.random.uniform(-90,90,600)
rand_lng = np.random.uniform(-180,180,600)
lat_lng = pd.DataFrame({'Latitude':rand_lat, 'Longitude':rand_lng})
lat_lng
# rand_lat

,Latitude,Longitude
0,55.204447,-107.727083
1,-47.209877,-12.276691
2,23.619771,99.648171
3,-69.070016,46.002049
4,-54.593496,66.738513
...,...,...
595,-18.172646,178.743377
596,-77.543849,-103.394712
597,-81.031316,-143.548627
598,-79.633524,1.128788


In [6]:
cities = []
countries = []

for i in range (0,len(lat_lng['Latitude'])):
    city = citipy.nearest_city(lat_lng['Latitude'][i], lat_lng['Longitude'][i])
    cities.append(city.city_name)
    countries.append(city.country_code)
    
lat_lng['City'] = cities
lat_lng['Country'] = countries

# Check for duplicated cities (True=1, False=0, added together)
lat_lng.duplicated().sum()

0

In [7]:
# Count number of cities
len(lat_lng)

600

In [8]:
lat_lng

,Latitude,Longitude,City,Country
0,55.204447,-107.727083,meadow lake,ca
1,-47.209877,-12.276691,jamestown,sh
2,23.619771,99.648171,simao,cn
3,-69.070016,46.002049,port alfred,za
4,-54.593496,66.738513,saint-philippe,re
...,...,...,...,...
595,-18.172646,178.743377,isangel,vu
596,-77.543849,-103.394712,punta arenas,cl
597,-81.031316,-143.548627,mataura,pf
598,-79.633524,1.128788,hermanus,za


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
requests.get('http://api.openweathermap.org/data/2.5/weather?q=london&appid=47e33a210410942f6c73a407a06be6e5')

<Response [200]>

In [21]:
# Create city data list to append to
city_data = []

# Print initial request
print('Beginning Data Retrieval')
print('-----------------------------')

# Create record counter
record_count = 1

# Loop through each city in lat_lng list
# if limited, stall code at each iteration (with time.sleep)
for x in lat_lng['City'][:]:
    try:
        response = requests.get(query_url+x).json()
        
        city = response['name']
        lon = response['coord']['lon']
        lat = response['coord']['lat']
        temp_max = response['main']['temp_max']
        humidity = response['main']['humidity']
        cloudiness = response['clouds']['all']
        wind_speed = response['wind']['speed']
        country = response['sys']['country']
        date = response['dt']
        
        city_data.append({'City':city,
                          'Longitude':lon,
                          'Latitude':lat,
                          'Max Temperature':temp_max,
                          'Humidity':humidity,
                          'Cloudiness':cloudiness,
                          'Wind Speed':wind_speed,
                          'Country':country,
                          'Date':date})
        
        # Indicate city has been loaded
        print("Processed Record %s | %s" % (record_count, city))
        
        # Add 1 to record count
        record_count = record_count + 1
        
    # If error occurs, skip city
    except:
        print("City not found. Skipping...")
        pass

# Indicate all data has been loaded
print('-----------------------------')
print('Data Retrieval Complete')
print('-----------------------------')

Beginning Data Retrieval
-----------------------------
Processed Record 1 | Meadow Lake
Processed Record 2 | Jamestown
Processed Record 3 | Simao
Processed Record 4 | Port Alfred
Processed Record 5 | Saint-Philippe
Processed Record 6 | Upernavik
Processed Record 7 | Saint-Philippe
Processed Record 8 | Rikitea
City not found. Skipping...
City not found. Skipping...
Processed Record 9 | Codrington
Processed Record 10 | Ngunguru
Processed Record 11 | Sinnamary
Processed Record 12 | Pacific Grove
Processed Record 13 | Shingū
Processed Record 14 | Bredasdorp
Processed Record 15 | Bashtanka
Processed Record 16 | Fortuna
Processed Record 17 | Lavrentiya
Processed Record 18 | Agadez
Processed Record 19 | Mataura
Processed Record 20 | Teahupoo
Processed Record 21 | Rikitea
Processed Record 22 | ‘Ewa Beach
Processed Record 23 | Hithadhoo
Processed Record 24 | Iqaluit
Processed Record 25 | Talnakh
Processed Record 26 | New Norfolk
Processed Record 27 | Inuvik
Processed Record 28 | Erenhot
Process

Processed Record 243 | Hay River
Processed Record 244 | Ossora
Processed Record 245 | Bemidji
City not found. Skipping...
Processed Record 246 | Dubovyy Umët
Processed Record 247 | Atuona
Processed Record 248 | Tuktoyaktuk
Processed Record 249 | Atuona
Processed Record 250 | Arraial do Cabo
Processed Record 251 | Avarua
Processed Record 252 | Dawson Creek
Processed Record 253 | Jacareacanga
Processed Record 254 | Punta Arenas
Processed Record 255 | Jamestown
City not found. Skipping...
Processed Record 256 | Carnarvon
Processed Record 257 | Warwick
Processed Record 258 | Codrington
Processed Record 259 | Dunedin
Processed Record 260 | Thompson
Processed Record 261 | Thompson
Processed Record 262 | Paamiut
Processed Record 263 | Klaksvík
Processed Record 264 | Muros
Processed Record 265 | Fréjus
Processed Record 266 | Likasi
Processed Record 267 | Aklavik
Processed Record 268 | Port Alfred
Processed Record 269 | Rikitea
Processed Record 270 | Ambāgarh Chauki
Processed Record 271 | Bengk

Processed Record 478 | Atar
Processed Record 479 | Butaritari
Processed Record 480 | Bluff
Processed Record 481 | Wanning
Processed Record 482 | São João da Barra
Processed Record 483 | Nouakchott
Processed Record 484 | Zhigalovo
Processed Record 485 | Jamestown
Processed Record 486 | Soc Trang
Processed Record 487 | Rikitea
City not found. Skipping...
Processed Record 488 | Esperance
Processed Record 489 | Ushuaia
Processed Record 490 | Mahébourg
Processed Record 491 | Praia da Vitória
Processed Record 492 | Te Anau
Processed Record 493 | San Felipe
Processed Record 494 | Hobart
Processed Record 495 | Kumasi
Processed Record 496 | Vaini
Processed Record 497 | Tuktoyaktuk
Processed Record 498 | Esperance
Processed Record 499 | Mataura
Processed Record 500 | Rikitea
Processed Record 501 | Bua Yai
Processed Record 502 | Broome
Processed Record 503 | Jamestown
Processed Record 504 | Butaritari
City not found. Skipping...
Processed Record 505 | Bengkulu
Processed Record 506 | Shingū
Proces

In [23]:
# Test the output of above
city_data

[{'City': 'Meadow Lake',
  'Longitude': -106.5436,
  'Latitude': 34.8014,
  'Max Temperature': 93.22,
  'Humidity': 20,
  'Cloudiness': 42,
  'Wind Speed': 6.31,
  'Country': 'US',
  'Date': 1628534576},
 {'City': 'Jamestown',
  'Longitude': -79.2353,
  'Latitude': 42.097,
  'Max Temperature': 89.02,
  'Humidity': 52,
  'Cloudiness': 20,
  'Wind Speed': 8.99,
  'Country': 'US',
  'Date': 1628534498},
 {'City': 'Simao',
  'Longitude': 100.9748,
  'Latitude': 22.7886,
  'Max Temperature': 67.23,
  'Humidity': 99,
  'Cloudiness': 100,
  'Wind Speed': 0.34,
  'Country': 'CN',
  'Date': 1628534992},
 {'City': 'Port Alfred',
  'Longitude': 26.891,
  'Latitude': -33.5906,
  'Max Temperature': 57.51,
  'Humidity': 70,
  'Cloudiness': 100,
  'Wind Speed': 20.09,
  'Country': 'ZA',
  'Date': 1628534688},
 {'City': 'Saint-Philippe',
  'Longitude': 55.7679,
  'Latitude': -21.3585,
  'Max Temperature': 69.39,
  'Humidity': 85,
  'Cloudiness': 95,
  'Wind Speed': 1.05,
  'Country': 'RE',
  'Date': 1

In [28]:
# Export to CSV

# Set keys
keys = city_data[0].keys()

# Use csv.DictWriter to write CSV
city_data_csv = open("city_data.csv", "w")
dict_writer = csv.DictWriter(city_data_csv, keys)
dict_writer.writeheader()
dict_writer.writerows(city_data)
city_data_csv.close()

In [24]:
pd.DataFrame(city_data)

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Meadow Lake,-106.5436,34.8014,93.22,20,42,6.31,US,1628534576
1,Jamestown,-79.2353,42.0970,89.02,52,20,8.99,US,1628534498
2,Simao,100.9748,22.7886,67.23,99,100,0.34,CN,1628534992
3,Port Alfred,26.8910,-33.5906,57.51,70,100,20.09,ZA,1628534688
4,Saint-Philippe,55.7679,-21.3585,69.39,85,95,1.05,RE,1628534580
...,...,...,...,...,...,...,...,...,...
554,Isangel,169.2667,-19.5500,66.31,82,40,4.61,VU,1628534786
555,Punta Arenas,-70.9167,-53.1500,46.51,66,40,23.02,CL,1628534656
556,Mataura,168.8643,-46.1927,40.48,97,70,4.54,NZ,1628534546
557,Hermanus,19.2345,-34.4187,51.91,68,5,4.09,ZA,1628534682


In [ ]:
response = requests.get(query_url+'london').json()
response['coord']['lon']
response['coord']['lat']
response['main']['temp_max']
response['main']['temp_min']


# response['weather'][0]['description']



In [10]:
requests.get(query_url+'london').json()

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 60.6,
  'feels_like': 60.64,
  'temp_min': 58.37,
  'temp_max': 64.18,
  'pressure': 1014,
  'humidity': 91},
 'visibility': 10000,
 'wind': {'speed': 9.22, 'deg': 240},
 'clouds': {'all': 86},
 'dt': 1628533565,
 'sys': {'type': 2,
  'id': 2019646,
  'country': 'GB',
  'sunrise': 1628483804,
  'sunset': 1628537722},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression